In [1]:
    import requests
    import pandas as pd
    from bs4 import BeautifulSoup
    from io import StringIO
    import time

    def get_day_price(code, max_pages=633):
        base_url = f"https://finance.naver.com/item/sise_day.naver?code={code}"
        headers = {"User-Agent": "Mozilla/5.0"}
        df_total = pd.DataFrame()

        for page in range(1, max_pages + 1):
            url = f"{base_url}&page={page}"
            res = requests.get(url, headers=headers)
            if res.status_code != 200:
                print(f"페이지 {page} 요청 실패: 상태 코드 {res.status_code}")
                break

            soup = BeautifulSoup(res.text, "html.parser")
            table = soup.find("table", class_="type2")
            if table is None:
                print(f"페이지 {page}에서 테이블을 찾을 수 없습니다.")
                break

            df = pd.read_html(StringIO(str(table)), header=0)[0]
            df = df.dropna()

            # 👉 만약 페이지에 데이터가 없는 경우 자동 종료
            if df.empty:
                print(f"\n✅ 데이터 없음: 페이지 {page}에서 종료합니다.")
                break

            # 전일비 처리
            def parse_change(val):
                val = str(val).strip().replace(",", "")
                if val.startswith("상승"):
                    return int(val.replace("상승", "").strip())
                elif val.startswith("하락"):
                    return -int(val.replace("하락", "").strip())
                elif val.startswith("보합"):
                    return 0
                else:
                    return None

            df["전일비"] = df["전일비"].apply(parse_change)

            # 거래량 처리
            df["거래량"] = df["거래량"].astype(str).str.replace(",", "")
            df["거래량"] = df["거래량"].apply(lambda x: int(float(x)))

            # ✅ 출력 – 첫 페이지만 헤더 포함
            if page == 1:
                print(df.to_string(index=False))  # 헤더 포함
            else:
                print(df.to_string(index=False, header=False))  # 헤더 생략

            df_total = pd.concat([df_total, df], ignore_index=True)
            time.sleep(0.5)

        return df_total

    # 종목 코드 및 기업명 설정
    code = "009540"
    company_name = "HD한국조선해양"

    # 데이터 수집
    df = get_day_price(code, max_pages=633)

    # 컬럼명 정리
    df.columns = ["날짜", "종가", "전일비", "시가", "고가", "저가", "거래량"]

    # 날짜 형식 변환
    df["날짜"] = pd.to_datetime(df["날짜"])

    # ✅ 날짜 기준으로 중복 제거
    df = df.drop_duplicates(subset="날짜", keep="first")

    # 날짜 내림차순 정렬
    df = df.sort_values("날짜", ascending=False)

    # CSV 파일로 저장
    filename = f"{company_name}.csv"
    df.to_csv(filename, index=False, encoding="utf-8-sig")
    print(f"\n✅ {filename} 파일이 저장되었습니다.")


        날짜       종가   전일비       시가       고가       저가    거래량
2025.04.25 265500.0 16000 258500.0 265500.0 255000.0 591526
2025.04.24 249500.0 16000 233500.0 251000.0 230500.0 638799
2025.04.23 233500.0  3000 233000.0 236500.0 229000.0 186906
2025.04.22 230500.0  8500 222000.0 232000.0 222000.0 180717
2025.04.21 222000.0 -4000 224500.0 227500.0 217500.0 163783
2025.04.18 226000.0 -4000 232000.0 233000.0 223000.0 121058
2025.04.17 230000.0  4000 226000.0 231000.0 225000.0 169471
2025.04.16 226000.0     0 228500.0 230000.0 224500.0 149399
2025.04.15 226000.0     0 224000.0 226500.0 222000.0 154973
2025.04.14 226000.0  5000 225500.0 228000.0 222000.0 254854
2025.04.11 221000.0   8000 218000.0 221500.0 215500.0 299160
2025.04.10 213000.0  17500 207000.0 213500.0 203500.0 262869
2025.04.09 195500.0   3600 197000.0 204000.0 192000.0 306003
2025.04.08 191900.0   5700 191300.0 195300.0 190500.0 320962
2025.04.07 186200.0 -17800 191100.0 196000.0 185700.0 303590
2025.04.04 204000.0  -3500 203500.0